In [1]:
## imports
import pandas as pd
import numpy as np


In [2]:
## read data and view first 5 rows
filepath = r"C:\Projects\Deep Learning Academy\PerimeterX\challenge.tsv"
data = pd.read_csv(filepath_or_buffer=filepath, delimiter="\t", header=None)
print(data.head(5))
print("\n The shape of the data is:")
print(data.shape)

            0               1                      2  \
0  1473756847   175.51.37.136  /product/3931/details   
1  1473756857  126.170.31.163                  /home   
2  1473756861   208.78.171.75     /search?query=1403   
3  1473756863  75.225.145.170              /checkout   
4  1473756863    35.106.55.96                /signup   

                                                   3  
0  Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.3...  
1  Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5)...  
2  Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5)...  
3  Mozilla/5.0 (Windows NT 6.3) AppleWebKit/537.3...  
4  Mozilla/5.0 (Windows NT 6.3) AppleWebKit/537.3...  

 The shape of the data is:
(94200, 4)


In [3]:
# The data is with a shape of (94200,4)
# Looking at the data, it seems that the first column is in chronological order - could be a time stamp
# We will check that this columns is in chronological order and convert it to pandas timestamp in case it is
is_chronological = np.all(data.iloc[:,0].diff()[1:] >= 0)
if is_chronological:
    data.iloc[:,0] = pd.to_datetime(data.iloc[:,0],unit='s')
print(data.iloc[:,0].head(5))


0   2016-09-13 08:54:07
1   2016-09-13 08:54:17
2   2016-09-13 08:54:21
3   2016-09-13 08:54:23
4   2016-09-13 08:54:23
Name: 0, dtype: datetime64[ns]


In [4]:
# First column is indeed in chronological order - timestamp
# Second column appears to be the ip address
# Third column appears to be the the customer request
# Fourth column appears to be some technical details about the customer software environment
## lets rename the columns for better code readability
data.columns = ["time", "ip", "request", "details"]

In [5]:
## lets break down the fourth column into different features and remove unnecessary ones
data[["firefox_ver", "details"]] = data["details"].str.split(n=1, expand=True)
data[["os", "details"]] = data["details"].str.split(")", n=1, expand=True)
data["os"] = data["os"].str.replace("(", "")
data[["apple_web_kit", "details"]] = data["details"].str.split("(", n=1, expand=True)
data[["html", "details"]] = data["details"].str.split(")", n=1, expand=True)
data[["chrome_ver", "details"]] = data["details"].str.split(n=1, expand=True)
data.rename({"details": "safari_ver"}, axis=1, inplace=True)
unique_cols = data.apply(pd.Series.nunique)

# remove columns that have only one unique value - these columns are not meaningful
cols_to_remove = [col for ind, col in enumerate(unique_cols.index) if unique_cols[ind] == 1]
data.drop(cols_to_remove, axis=1, inplace=True)
print(data.columns)

Index(['time', 'ip', 'request', 'os', 'chrome_ver'], dtype='object')


In [6]:
# The only information that is meaningful is the "os" (operating system) and "chrome_ver" (chrome version)
# we will concatenate them back into detailsdata["details"] = data["os"] + "_" + data["chrome_ver"]
data["details"] = data["os"] + "_" + data["chrome_ver"]
data.drop(["os", "chrome_ver"], axis=1, inplace=True)

In [7]:
## Check if there are missing values and fill/drop them
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94200 entries, 0 to 94199
Data columns (total 4 columns):
time       94200 non-null datetime64[ns]
ip         94200 non-null object
request    94200 non-null object
details    94200 non-null object
dtypes: datetime64[ns](1), object(3)
memory usage: 2.9+ MB


In [8]:
# There are no missing values
## examining the time column
time_hist = data["time"].value_counts().sort_values(ascending=False)
print(time_hist.head(30))
print(time_hist.shape)

2016-09-13 18:20:07    2
2016-09-13 11:53:05    2
2016-09-15 18:07:57    2
2016-09-17 17:06:37    2
2016-09-13 12:25:35    2
2016-09-18 15:11:17    2
2016-09-15 00:22:43    2
2016-09-16 18:28:09    2
2016-09-17 15:36:12    2
2016-09-13 11:52:43    2
2016-09-18 18:34:15    2
2016-09-18 05:10:52    2
2016-09-14 05:54:29    2
2016-09-18 08:23:35    2
2016-09-13 12:30:27    2
2016-09-14 16:10:04    2
2016-09-18 22:46:47    2
2016-09-19 09:52:40    2
2016-09-13 14:43:04    2
2016-09-15 15:54:22    2
2016-09-19 16:34:36    2
2016-09-14 11:18:14    2
2016-09-15 14:03:57    2
2016-09-14 16:23:55    2
2016-09-16 04:23:04    2
2016-09-13 12:29:59    2
2016-09-20 02:29:46    2
2016-09-19 08:32:24    2
2016-09-20 02:00:03    2
2016-09-16 13:46:15    2
Name: time, dtype: int64
(93573,)


In [9]:
# There are no more than two requests at the same time stamp
## examining the ip column
ip_hist = data["ip"].value_counts().sort_values(ascending=False)
print(ip_hist.head(30))
print(ip_hist.shape)

35.106.55.96       1547
233.74.135.80      1453
166.164.14.1        520
41.12.245.38        480
230.25.250.72        61
126.87.50.48         56
142.200.200.107      44
46.172.26.84         39
210.41.151.100       25
117.246.221.124      22
64.223.217.72        21
50.76.76.111         21
76.58.197.12         20
151.179.104.109      20
222.6.181.228        20
141.57.7.72          20
136.172.68.175       20
163.242.119.53       20
114.54.190.111       20
244.26.45.76         19
108.74.164.157       19
158.167.171.198      19
128.40.245.181       19
161.85.35.229        19
5.34.253.135         19
142.55.42.221        19
107.185.105.78       19
201.20.6.220         19
210.48.104.83        19
165.153.129.77       19
Name: ip, dtype: int64
(10006,)


In [10]:
## We can see that there are some ips with significant more traffic than the others - these ips are suspicious to be attackers
## examining the request column
request_hist = data["request"].value_counts().sort_values(ascending=False)
print(request_hist.head(10))
print(request_hist.shape)

/contact                 6332
/home                    6304
/view_order              6242
/signup                  6141
/checkout                6081
/search?query=7           362
/search?query=88          329
/search?query=13          318
/search?query=3265         11
/product/8194/details      11
Name: request, dtype: int64
(19054,)


In [11]:
# We can see that there are 8 requests that occur much more than others
## lets find how many different queries and products there are in the traffic data
# and see if there is any ip that tried to retrieve all this data
query_inds = data["request"].str.contains("query")
num_unique_queries = len(data["request"].loc[query_inds].unique())
product_inds = data["request"].str.contains("product")
num_unique_products = len(data["request"].loc[product_inds].unique())
print("The number of unique queries is: " + str(num_unique_queries))
print("The number of unique products is: " + str(num_unique_products))


The number of unique queries is: 9528
The number of unique products is: 9521


In [12]:
# There is no ip that appears close to these numbers.
## examining the details column
details_hist = data["details"].value_counts().sort_values(ascending=False)
print(details_hist.head(50))
print(details_hist.shape)


Windows NT 6.1_Chrome/45.0.2454.85                       2481
Macintosh; Intel Mac OS X 10_8_3_Chrome/41.0.2403.157    2167
Macintosh; Intel Mac OS X 10_9_5_Chrome/39.0.2403.157    2134
Macintosh; Intel Mac OS X 10_7_4_Chrome/42.0.2403.157    2118
Macintosh; Intel Mac OS X 10_7_4_Chrome/43.0.2403.157    2101
Macintosh; Intel Mac OS X 10_9_5_Chrome/45.0.2403.157    2093
Macintosh; Intel Mac OS X 10_8_3_Chrome/38.0.2403.157    2070
Macintosh; Intel Mac OS X 10_9_5_Chrome/41.0.2403.157    2010
Macintosh; Intel Mac OS X 10_8_3_Chrome/43.0.2403.157    2009
Macintosh; Intel Mac OS X 10_7_4_Chrome/38.0.2403.157    1994
Macintosh; Intel Mac OS X 10_9_5_Chrome/44.0.2403.157    1980
Macintosh; Intel Mac OS X 10_8_3_Chrome/39.0.2403.157    1974
Macintosh; Intel Mac OS X 10_7_4_Chrome/39.0.2403.157    1961
Macintosh; Intel Mac OS X 10_8_3_Chrome/42.0.2403.157    1956
Macintosh; Intel Mac OS X 10_9_5_Chrome/38.0.2403.157    1940
Macintosh; Intel Mac OS X 10_7_4_Chrome/44.0.2403.157    1929
Macintos

In [13]:
# There is nothing much to say about the occurrence of the different software specifications
##  we will check the top 10 ips found in the data and consider them as suspicious ips
# we will examine each one of them and find the attackers and the type of the attack
ips_to_check = ip_hist.index[:10]
insights = pd.DataFrame(index=ips_to_check, columns=["ip_appearance", "unique_requests", "unique_details",\
                                                     "mean_sec_diff" ,"contact", "home",\
                                                     "view_order", "signup", "checkout",\
                                                     "query7", "query88", "query13", "product", "std_sec"])
for row, suspicious_ip in enumerate(ips_to_check):
    suspicious_data = data[data["ip"] == suspicious_ip]
    insights["ip_appearance"][row] = suspicious_data.shape[0]
    insights["unique_requests"][row] = len(suspicious_data["request"].unique())
    insights["unique_details"][row] = len(suspicious_data["details"].unique())
    insights["mean_sec_diff"][row] = suspicious_data["time"].diff().mean().total_seconds()
    insights["contact"][row] = suspicious_data["request"].str.contains("contact").sum()
    insights["home"][row] = suspicious_data["request"].str.contains("home").sum()
    insights["view_order"][row] = suspicious_data["request"].str.contains("view_order").sum()
    insights["signup"][row] = suspicious_data["request"].str.contains("signup").sum()
    insights["checkout"][row] = suspicious_data["request"].str.contains("checkout").sum()
    insights["query7"][row] = suspicious_data["request"].str.contains("query=7").sum()
    insights["query88"][row] = suspicious_data["request"].str.contains("query=88").sum()
    insights["query13"][row] = suspicious_data["request"].str.contains("query=13").sum()
    insights["product"][row] = suspicious_data["request"].str.contains("product").sum()
    insights["std_sec"][row] = np.std(suspicious_data['time'].astype(str).str.split(':', expand=True).iloc[:,-1].astype(int))
print(insights)

                ip_appearance unique_requests unique_details mean_sec_diff  \
35.106.55.96             1547            1001             56       382.776   
233.74.135.80            1453             962             56         407.7   
166.164.14.1              520               3              1        6.8921   
41.12.245.38              480               3              1       7.50522   
230.25.250.72              61               1              1            98   
126.87.50.48               56              56             38         10301   
142.200.200.107            44              44             31       13398.7   
46.172.26.84               39               1              1       153.158   
210.41.151.100             25              20              1       21663.8   
117.246.221.124            22              21              1       25393.2   

                contact home view_order signup checkout query7 query88  \
35.106.55.96        108  122        103     94      100     60     

In [ ]:
## Looking at the insights dataframe, we can see that ips 35.106.55.96 and 233.74.135.80  have significant more
# requests of type contact, home, view order, signup, checkout and product. These requests
# are from 56 different operating systems. The attack seems to try and collect data of the products

## Looking at ips 166.164.14.1 and 41.12.245.38, the average time between two requests is significantly lower
# and there are three unique requests of queries 7, 13 and 88. It seems that the atack in this case is
# denial of service - requesting these queries many times in a short amount of time

## Looking at ips 230.25.250.72 and 46.172.26.84, we can see that all the requests are at the same
# (standard deviation of the seconds is 0) and that all the request are of type home.
